<a href="https://colab.research.google.com/github/CHATHARABOINASAGAR/NLP-LAB/blob/main/NLP-12-09-2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, MaxPooling1D, Flatten, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

df = pd.read_csv("/content/train.csv")
print(df.head())

texts = df["text"].values
labels = df["target"].values

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in STOPWORDS]
    return " ".join(tokens)

df["clean_text"] = df["text"].apply(clean_text)

X_train, X_val, y_train, y_val = train_test_split(df["clean_text"], labels, test_size=0.2, random_state=42)

count_vec = CountVectorizer()
X_train_count = count_vec.fit_transform(X_train)
X_val_count = count_vec.transform(X_val)

tfidf_vec = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vec.fit_transform(X_train)
X_val_tfidf = tfidf_vec.transform(X_val)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_val_tfidf)

svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)
y_pred_svm = svm.predict(X_val_tfidf)

print("\n--- Logistic Regression (TF-IDF) ---")
print(classification_report(y_val, y_pred_lr))

print("\n--- SVM (TF-IDF) ---")
print(classification_report(y_val, y_pred_svm))

tokenizer = Tokenizer(num_words=10000, oov_token="")
tokenizer.fit_on_texts(df["clean_text"])
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

max_len = 30
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post")
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding="post")

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

mlp = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])
mlp.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
mlp.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_mlp = (mlp.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- MLP (Embeddings) ---")
print(classification_report(y_val, y_pred_mlp))

cnn = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    Conv1D(128, 5, activation="relu"),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])
cnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
cnn.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_cnn = (cnn.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- CNN (Embeddings) ---")
print(classification_report(y_val, y_pred_cnn))

lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(128, dropout=0.3, recurrent_dropout=0.3),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])
lstm.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
lstm.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_val_pad, y_val))

y_pred_lstm = (lstm.predict(X_val_pad) > 0.5).astype("int32")

print("\n--- LSTM (Embeddings) ---")
print(classification_report(y_val, y_pred_lstm))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  

--- Logistic Regression (TF-IDF) ---
              precision    recall  f1-score   support

           0       0.79      0.89      0.84       874
           1       0.82      0.68      0.74       649

    accuracy                           0.80      1523
   macro avg       0.80      0.79      0.79      1523
weighted avg       0.80      0.80      0.80      1523


--- SVM (TF-IDF) ---
              precision    recall  f1-score   support

           0       0.79

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.5707 - loss: 0.6788 - val_accuracy: 0.6704 - val_loss: 0.6105
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.7723 - loss: 0.5268 - val_accuracy: 0.7406 - val_loss: 0.5000
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.8616 - loss: 0.3450 - val_accuracy: 0.7978 - val_loss: 0.4578
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.8963 - loss: 0.2748 - val_accuracy: 0.7571 - val_loss: 0.5203
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9077 - loss: 0.2311 - val_accuracy: 0.7932 - val_loss: 0.5392
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

--- MLP (Embeddings) ---
              precision    recall  f1-score   support

           0       0.78      0.90      0.83       874
           1       0.83      0.65      0.73       649

    accuracy                           0.79      1523
   macro avg       0.80      0.77      0.78      1523
weighted avg       0.80

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


191/191 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.6252 - loss: 0.6384 - val_accuracy: 0.7932 - val_loss: 0.4576
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.8758 - loss: 0.3119 - val_accuracy: 0.7886 - val_loss: 0.5196
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9480 - loss: 0.1455 - val_accuracy: 0.7544 - val_loss: 0.6524
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9690 - loss: 0.0864 - val_accuracy: 0.7584 - val_loss: 0.7532
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9772 - loss: 0.0567 - val_accuracy: 0.7728 - val_loss: 0.8521
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

--- CNN (Embeddings) ---
              precision    recall  f1-score   support

           0       0.78      0.84      0.81       874
           1       0.76      0.69      0.72       649

    accuracy                           0.77      1523
   macro avg       0.77      0.76      0.76      1523
weighted avg       0.77      0.77

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


191/191 ━━━━━━━━━━━━━━━━━━━━ 24s 103ms/step - accuracy: 0.5545 - loss: 0.6879 - val_accuracy: 0.5837 - val_loss: 0.6576
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 19s 96ms/step - accuracy: 0.7557 - loss: 0.5313 - val_accuracy: 0.7695 - val_loss: 0.4818
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 18s 96ms/step - accuracy: 0.8555 - loss: 0.3841 - val_accuracy: 0.7774 - val_loss: 0.5218
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 19s 90ms/step - accuracy: 0.9047 - loss: 0.3043 - val_accuracy: 0.7748 - val_loss: 0.5350
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 18s 94ms/step - accuracy: 0.9152 - loss: 0.2513 - val_accuracy: 0.7715 - val_loss: 0.5695
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step

--- LSTM (Embeddings) ---
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       874
           1       0.75      0.70      0.72       649

    accuracy                           0.77      1523
   macro avg       0.77      0.76      0.76      1523
weighted avg       0.77  